In [1]:
# 1. INSTALAR LIBRERÍAS
!pip install transformers datasets accelerate -Uq

import os
import gc
import torch
import shutil
from datasets import load_dataset
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments, DataCollatorForLanguageModeling

# ==========================================
# 2. CONFIGURACIÓN Y RUTAS
# ==========================================
# Ajusta esto según dónde subiste tus archivos en Kaggle
BASE_PATH = "/kaggle/input/arrows-and-boxes" 

# Mapeo de Expertos
TRAIN_FILES = {
    "BOX":   f"{BASE_PATH}/dataset_boxes.jsonl",
    "ARROW": f"{BASE_PATH}/dataset_arrows.jsonl"
}

# Verificación
for k, v in TRAIN_FILES.items():
    if os.path.exists(v):
        print(f"✅ {k}: Encontrado ({os.path.getsize(v)/1024:.2f} KB)")
    else:
        print(f"❌ {k}: NO ENCONTRADO en {v} (Revisa la ruta)")

# ==========================================
# 3. DICCIONARIO MAESTRO V18 (ARCHITECT)
# ==========================================
MODEL_CHECKPOINT = "gpt2" # GPT-2 Small (124M)

SPECIAL_TOKENS = {
    "pad_token": "[PAD]",
    "additional_special_tokens": [
        # Control
        "[START]", "[STOP]", "[VOID]", 
        # Estructura Híbrida
        "░", 
        "<L01>", "<L02>", "<L03>", "<L04>", "<L05>", "<L06>", "<L07>", "<L08>", "<L09>", "<L10>",
        "[S:00]", # En entrenamiento siempre es 0, pero lo mantenemos por compatibilidad
        # Anchors y Primitivas de Caja
        "N", "S", "E", "W", 
        "+", "-", "|",
        # Flechas
        "^", "v", "<", ">",
        # Tags del Prompt
        "[TYPE:BOX]", "[TYPE:ARROW]",
        "[STYLE:SOLID]",
        "[DIR:UP]", "[DIR:DOWN]", "[DIR:LEFT]", "[DIR:RIGHT]",
        # Dimensiones (Importante: El modelo tokenizará los números, pero enseñamos el formato)
        "[DIM:", "[LEN:" 
    ]
}

# ==========================================
# 4. MOTOR DE ENTRENAMIENTO (The Gym)
# ==========================================
def train_architect(expert_name, jsonl_path):
    print(f"\n▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬")
    print(f"🏗️  CONSTRUYENDO EXPERTO: {expert_name}")
    print(f"▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬")
    
    output_dir = f"./expert_{expert_name.lower()}"
    
    # Tokenizer
    tokenizer = GPT2Tokenizer.from_pretrained(MODEL_CHECKPOINT)
    tokenizer.add_special_tokens(SPECIAL_TOKENS)
    
    # Dataset
    dataset = load_dataset("json", data_files=jsonl_path, split="train")
    
    def format_fn(x):
        text = f"{x['prompt']} {x['completion']}"
        return tokenizer(text, truncation=True, padding="max_length", max_length=128) # Subimos a 128 por si las cajas son grandes
    
    tokenized = dataset.map(format_fn, batched=False, remove_columns=dataset.column_names)
    
    # Modelo
    model = GPT2LMHeadModel.from_pretrained(MODEL_CHECKPOINT)
    model.resize_token_embeddings(len(tokenizer))
    
    # Argumentos (Overfitting táctico para precisión geométrica)
    args = TrainingArguments(
        output_dir=output_dir,
        overwrite_output_dir=True,
        num_train_epochs=10,          # 10 Vueltas es un buen punto de partida
        per_device_train_batch_size=32, 
        learning_rate=5e-4,
        save_strategy="no",
        logging_steps=50,
        report_to="none"
    )
    
    trainer = Trainer(
        model=model, 
        args=args, 
        train_dataset=tokenized, 
        data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False)
    )
    
    trainer.train()
    
    # Guardar
    print(f"💾 Guardando planos en {output_dir}...")
    model.save_pretrained(output_dir)
    tokenizer.save_pretrained(output_dir)
    
    # Limpiar
    del model, trainer, tokenizer
    torch.cuda.empty_cache()
    gc.collect()

# --- EJECUCIÓN ---
# Solo entrenamos si encontró los archivos
if os.path.exists(TRAIN_FILES["BOX"]):
    train_architect("BOX", TRAIN_FILES["BOX"])

if os.path.exists(TRAIN_FILES["ARROW"]):
    train_architect("ARROW", TRAIN_FILES["ARROW"])

# ==========================================
# 5. EMPAQUETADO (Delivery)
# ==========================================
print("\n📦 EMPAQUETANDO HERRAMIENTAS...")
from IPython.display import FileLink

zip_name = "ASCII_Architect_V1_Models"
os.makedirs("Architect_Package", exist_ok=True)

for folder in ["expert_box", "expert_arrow"]:
    if os.path.exists(folder):
        shutil.move(folder, f"Architect_Package/{folder}")

shutil.make_archive(zip_name, 'zip', "Architect_Package")
print(f"✅ ¡LISTO! Descarga tus arquitectos:")
FileLink(f"{zip_name}.zip")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 105.7 MB/s eta 0:00:0000:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.9/380.9 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 41.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 74.3 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 120.6 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 92.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5

2025-12-14 18:38:00.437505: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765737480.648310      47 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765737480.708906      47 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

❌ BOX: NO ENCONTRADO en /kaggle/input/dataset-v18-architect/dataset_boxes.jsonl (Revisa la ruta)
❌ ARROW: NO ENCONTRADO en /kaggle/input/dataset-v18-architect/dataset_arrows.jsonl (Revisa la ruta)

📦 EMPAQUETANDO HERRAMIENTAS...
✅ ¡LISTO! Descarga tus arquitectos:


/kaggle/working/ASCII_Architect_V1_Models.zip

In [ ]:
# 1. INSTALAR LIBRERÍAS (Incluyendo el parche de protobuf)
!pip install transformers datasets accelerate protobuf -Uq

import os
import time

print("✅ Librerías instaladas.")
print("🔄 REINICIANDO KERNEL AUTOMÁTICAMENTE PARA APLICAR CAMBIOS...")
# Esto mata la sesión actual y arranca una limpia con las librerías nuevas
os.kill(os.getpid(), 9)

In [1]:
import os

# CONFIGURACIÓN DE RUTAS
# Basado en tu captura, tu dataset se llama "arrows-and-boxes"
BASE_PATH = "/kaggle/input/arrows-and-boxes"

FILES = {
    "BOX":   f"{BASE_PATH}/dataset_boxes.jsonl",
    "ARROW": f"{BASE_PATH}/dataset_arrows.jsonl"
}

print("🕵️ INSPECCIONANDO RUTAS...")

all_good = True
for name, path in FILES.items():
    if os.path.exists(path):
        size = os.path.getsize(path) / 1024
        print(f"   ✅ {name}: ENCONTRADO ({size:.2f} KB) -> {path}")
    else:
        print(f"   ❌ {name}: NO EXISTE en {path}")
        all_good = False

if not all_good:
    print("\n⚠️ ALERTA: Revisa el nombre de la carpeta en el panel derecho (Input).")
    print("Contenido real de /kaggle/input:")
    try:
        print(os.listdir("/kaggle/input"))
        print(f"Contenido de {BASE_PATH}:")
        print(os.listdir(BASE_PATH))
    except:
        print("No pude leer los directorios.")
else:
    print("\n🚀 TODO LISTO. PUEDES PASAR A ENTRENAR.")

🕵️ INSPECCIONANDO RUTAS...
   ✅ BOX: ENCONTRADO (1708.92 KB) -> /kaggle/input/arrows-and-boxes/dataset_boxes.jsonl
   ✅ ARROW: ENCONTRADO (644.83 KB) -> /kaggle/input/arrows-and-boxes/dataset_arrows.jsonl

🚀 TODO LISTO. PUEDES PASAR A ENTRENAR.


In [2]:
import gc
import torch
import shutil
from datasets import load_dataset
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments, DataCollatorForLanguageModeling

# 1. DICCIONARIO MAESTRO V18 (Architect)
MODEL_CHECKPOINT = "gpt2"

SPECIAL_TOKENS = {
    "pad_token": "[PAD]",
    "additional_special_tokens": [
        "[START]", "[STOP]", "[VOID]", 
        "░", 
        "<L01>", "<L02>", "<L03>", "<L04>", "<L05>", "<L06>", "<L07>", "<L08>", "<L09>", "<L10>",
        "[S:00]", 
        "N", "S", "E", "W", "+", "-", "|", "^", "v", "<", ">",
        "[TYPE:BOX]", "[TYPE:ARROW]", "[STYLE:SOLID]",
        "[DIR:UP]", "[DIR:DOWN]", "[DIR:LEFT]", "[DIR:RIGHT]", "[DIM:", "[LEN:" 
    ]
}

# 2. FUNCIÓN DE ENTRENAMIENTO
def train_architect(expert_name, jsonl_path):
    print(f"\n🏗️  CONSTRUYENDO EXPERTO: {expert_name}")
    
    output_dir = f"./expert_{expert_name.lower()}"
    
    tokenizer = GPT2Tokenizer.from_pretrained(MODEL_CHECKPOINT)
    tokenizer.add_special_tokens(SPECIAL_TOKENS)
    
    dataset = load_dataset("json", data_files=jsonl_path, split="train")
    
    def format_fn(x):
        text = f"{x['prompt']} {x['completion']}"
        return tokenizer(text, truncation=True, padding="max_length", max_length=128)
    
    tokenized = dataset.map(format_fn, batched=False, remove_columns=dataset.column_names)
    
    model = GPT2LMHeadModel.from_pretrained(MODEL_CHECKPOINT)
    model.resize_token_embeddings(len(tokenizer))
    
    args = TrainingArguments(
        output_dir=output_dir,
        overwrite_output_dir=True,
        num_train_epochs=10,
        per_device_train_batch_size=32,
        learning_rate=5e-4,
        save_strategy="no",
        logging_steps=50,
        report_to="none"
    )
    
    trainer = Trainer(model=model, args=args, train_dataset=tokenized, data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False))
    trainer.train()
    
    print(f"💾 Guardando {expert_name}...")
    model.save_pretrained(output_dir)
    tokenizer.save_pretrained(output_dir)
    
    del model, trainer, tokenizer
    torch.cuda.empty_cache()
    gc.collect()

# 3. EJECUTAR (Usando las rutas validadas en Celda 2)
# Nota: FILES viene de la celda anterior
train_architect("BOX", FILES["BOX"])
train_architect("ARROW", FILES["ARROW"])

print("\n✨ ENTRENAMIENTO COMPLETADO.")

2025-12-14 18:41:22.182755: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765737682.204504     160 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765737682.211060     160 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

...

💾 Guardando BOX...

🏗️  CONSTRUYENDO EXPERTO: ARROW


Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Step,Training Loss
50,1.817300
100,0.345600
150,0.214600
200,0.180500
250,0.165300
300,0.140300
350,0.117600
400,0.108500
450,0.107200
500,0.107200


💾 Guardando ARROW...

✨ ENTRENAMIENTO COMPLETADO.


In [3]:
from IPython.display import FileLink
import os
import shutil

zip_name = "ASCII_Architect_V1_Models"
os.makedirs("Architect_Package", exist_ok=True)

for folder in ["expert_box", "expert_arrow"]:
    if os.path.exists(folder):
        shutil.move(folder, f"Architect_Package/{folder}")

shutil.make_archive(zip_name, 'zip', "Architect_Package")
print(f"✅ DESCARGA LISTA:")
FileLink(f"{zip_name}.zip")

✅ DESCARGA LISTA:


/kaggle/working/ASCII_Architect_V1_Models.zip